In [ ]:
# -*- coding: utf-8 -*-
"""
SVM三分类完整代码
步骤：
1. 数据加载与检查
2. 数据预处理
3. 模型训练与调参
4. 模型评估
5. 结果可视化（可选）
"""

# 基础库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Sklearn模块
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    ConfusionMatrixDisplay,
)
from sklearn.pipeline import Pipeline

# 忽略警告
import warnings
warnings.filterwarnings('ignore')

# ---------------------- 1. 加载您的数据 ----------------------
# 替换以下示例数据为您的实际数据
# 假设数据格式：CSV文件，最后一列为标签列(0,1,2)，其余为特征
# 示例数据（使用sklearn自带的鸢尾花数据集演示数据加载逻辑）
from sklearn.datasets import load_iris
iris = load_iris()
X, y = iris.data, iris.target

# 您的数据加载示例（取消注释使用）：
# data = pd.read_csv('your_data.csv')
# X = data.iloc[:, :-1].values  # 特征：所有列除了最后一列
# y = data.iloc[:, -1].values   # 标签：最后一列

# 数据检查
print(f"特征形状: {X.shape}, 标签形状: {y.shape}")
print("类别分布:", np.bincount(y))

# ---------------------- 2. 数据预处理 ----------------------
# 划分训练集/测试集
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    stratify=y,  # 保持类别分布
    random_state=42
)

# 标准化处理
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ---------------------- 3. 模型训练与调参 ----------------------
# 定义管道（标准化+SVM）
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC())
])

# 超参数网格
param_grid = {
    'svm__C': [0.1, 1, 10],          # 正则化参数
    'svm__gamma': ['scale', 'auto', 0.1, 0.01],  # 核函数系数
    'svm__kernel': ['rbf', 'linear'], # 核函数类型
    'svm__decision_function_shape': ['ovo', 'ovr']  # 多分类策略
}

# 网格搜索
grid = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,
    n_jobs=-1,
    verbose=1
)

grid.fit(X_train, y_train)

# 输出最佳参数
print("\n最佳参数组合:", grid.best_params_)
print("最佳交叉验证准确率:", grid.best_score_)

# ---------------------- 4. 模型评估 ----------------------
# 使用最佳模型预测
best_model = grid.best_estimator_
y_pred = best_model.predict(X_test)

# 评估指标
print("\n测试集准确率:", accuracy_score(y_test, y_pred))
print("分类报告:\n", classification_report(y_test, y_pred))

# 混淆矩阵
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

# ---------------------- 5. 特征重要性分析（可选） ----------------------
# 注意：仅适用于线性核
if best_model.named_steps['svm'].kernel == 'linear':
    coef = best_model.named_steps['svm'].coef_
    print("\n特征重要性（权重）:")
    for i, class_coef in enumerate(coef):
        print(f"类别 {i} 的特征权重:", np.abs(class_coef))
else:
    print("\n提示：非线形核函数无法直接获取特征权重")

# ---------------------- 使用说明 ----------------------
"""
使用方法：
1. 将您的数据保存为CSV格式，确保最后一列是类别标签(0,1,2)
2. 替换代码中的示例数据加载部分（注释掉鸢尾花数据集，取消注释您的数据加载代码）
3. 运行完整代码

注意事项：
- 如果出现内存不足，可以减小param_grid中的参数组合
- 如果类别不平衡，在SVC参数中添加class_weight='balanced'
- 如果特征维度高，建议在Pipeline中添加PCA降维步骤
"""